Load dataset:

In [1]:
from HumanGenomeDataset.load_dataset import load_dataset

data_df = load_dataset('dna_protein_coding_sequences')
data_df.head(2)

,ID,sequence,category
0,NM_001368254.1,GCTGAGCTGAGCTGGGGCGCAGCCGCCTGTCTGCACCGGCAGCACC...,NM
1,NM_001350977.1,TCCACAACTGAAACATCCACTTCTGAACACCATGTCCTACTACAGC...,NM


Start simulation:

In [2]:
from src.simulation import ProteinSinthesisProcess
SIM_TIME = 30

In [3]:
protein_synthesis_process = ProteinSinthesisProcess(dna_sequences_df=data_df, verbose=True)
protein_synthesis_process.run(simulation_time=SIM_TIME) # run the simulation

Simulation environment initialized 	
Simulation started: 	 	
Time 2.1872589772973807: Protein synthesis started
Time 2.1872589772973807: Transcription started
Time 2.1872589772973807: Transcription ended
mRNA synthesized: 2
Time 2.1872589772973807: Translation started
Time 2.1872589772973807: Translation ended
Protein synthesized: 2 	 	
Time 8.249454904209527: Protein synthesis started
Time 8.249454904209527: Transcription started
Time 8.249454904209527: Transcription ended
mRNA synthesized: 4
Time 8.249454904209527: Translation started
Time 8.249454904209527: Translation ended
Protein synthesized: 4 	 	
Time 15.752681137778563: Protein synthesis started
Time 15.752681137778563: Transcription started
Time 15.752681137778563: Transcription ended
mRNA synthesized: 19
Time 15.752681137778563: Translation started
Time 15.752681137778563: Translation ended
Protein synthesized: 19 	 	
Time 16.527351152418955: Protein synthesis started
Time 16.527351152418955: Transcription started
Time 16.52

Results:

In [4]:
results_df = protein_synthesis_process.dna_sequences_df

In [5]:
print('Number of DNA sequence traslated:', results_df[results_df['mrna_sequences'].notna()].shape[0])
print('Number of DNA sequence not traslated:', results_df[results_df['protein_synthesized']==False].shape[0])

Number of DNA sequence traslated: 4
Number of DNA sequence not traslated: 1


In [6]:
results_df[results_df['mrna_sequences'].notna()][[
    'polypeptides_chains', 'number_of_proteins_synthesized']]

,polypeptides_chains,number_of_proteins_synthesized
47896,"[NH2-LFPFNSL-COOH, NH2-TRTLFIKLNL-COOH, NH2-AL...",6
85220,"[NH2-TF-COOH, NH2--COOH, NH2--COOH, NH2-EYLQQL...",4
164659,"[NH2--COOH, NH2-IFC-COOH]",2
172665,"[NH2--COOH, NH2-F-COOH, NH2-SLRPKR-COOH, NH2--...",19
